# Условия

В качестве технического тестового задания предлагалось:
    
1. Выбрать статью из списка
2. Реализовать подход из статьи
3. Провести эксперимент на реальных данных
4. Написать (этот) отчёт о задаче, методе, его особенностях, полезности, смысле результатов, перспективах

 ---
 

# Предложенные статьи

\+ их краткий обзор

### [MelGAN](https://arxiv.org/pdf/1910.06711.pdf)

Преложили быструю параллельную модель улучшения аудиосигнала в виде спектрограммы с помощью GAN'а. Подчеркнули недостатки существующих SOTA-решений (WORLD, Char2Wav - возвращали спектрограмму, а не аудиосигнал, Wavenet, SampleRNN, WaveRNN - были авторегрессионными, а поэтому и медленными), показали чем вдохновлялись (parallel WaveNet - оказалось очень красивой идеей, Clarinet использовали Normalizing Flows). Сопоставили вышеуказанное, добавили weightnorm и завернули всё в adversarial loss с тройным дискриминатором - получилось вкусно.  

В результате получили модель в 10раз быстрее конкурентов схожего качества, почти end-to-end, realtime inference.

### [Adversarial attack on Alexa](http://papers.nips.cc/paper/9362-adversarial-music-real-world-audio-adversary-against-wake-word-detection-system.pdf)

Предложили первую не-whitebox adversarial аудио атаку. Также замаскировали её под музыку, показали способность модели обманывать wake-word системы (на примере Amazon Alexa) не только виртуально, но и при звучании атаки в воздухе. Фактически они заставляли Алексу зависнуть тем, что не позволяли модели корректно распознать wake-word от владельца. Понизили F-меру распознавания ww Алексы с 93.4% to 11.0%. Более того, человек не заметит ататку так как она не слышна из-за музыки, под которую модель учится маскировать атаку.

Особенностью было то, что модель распознавания Алексы не была точно известна (обычно атаки проводят на точных копиях модели), а также для сбора датасета понадобилось всего 10 различных голосов, озвучивших ww с разными интонациями. Можно показать демку на Алисе.

### [Gradient isolated learning representations](https://arxiv.org/pdf/1905.11786v2.pdf)

Авторы предложили улучшение unsupervised подхода Contrastive Prediction Coding (CPC), чем почти не подняли качество распознавания звуков и спикеров, но значительно сократили вычислительные ресурсы на обучение модели. Опираясь на статьи о принципах работы мозга, утверждающих, что мозг максимизирует взаимную информацию разных частей одного объекта и минимизирует её для разных объектов, дополнили принципом иерархии репрезентаций: на каждом следующем слое сети за счёт жадного обучения должны создаваться более сложные признаки объекта в пределах рецептивной области.

В результате авторы разбили CPC модель по слоям на модули, добавив в каждый из которых Noise Contrastive Estimation loss. Благодаря этому модель может учиться итеративно, слой за слоем, значительно экономя процессорное время (не вычисляет последующие слои, пока не обучила предыдущие) и память (не протаскивает градиенты через всю модель, градиенты текут внутри только одного обучающегося блока)

### [GAN-based Speech Enhancemen](https://arxiv.org/pdf/1905.04874.pdf)

Авторы утверждают, что adversarial loss плох в задачах с улучшением аудио тк оценки дискриминатора бинарны, а L1 и L2 loss’ы не избавляют от артефактов аудио и не приближают предсказания модели к идеалу. Предложили способ сделать loss дискриминатора непрерывным для непрерывных метрик. Достигают это методом “сходного поведения” - минимизируют разницу \$(D(G(z), y) - Metric(G(z), y))^2\$ , где D измеряет “реалистичность” дорожки. Рассматривают подход на примере метрик PESQ и STOI. Показывают ограничения метода на тренировку с несколькими целевыми функциями.

### [U-Net for Speech Enhancement](https://openreview.net/pdf?id=SkeRTsAcYm)

Предложили использовать комплексные числа в в алгебраической форме в фильтрах свёрточной сети, взяв за основу сети архитектуру U-Net. Поменяли лосс из предыдущей работы, доработали его, добавив взевешивание по мощности шума и аудиодорожки. Улучшили метрики в среднем на 3%. 

### [ShaRNN](http://papers.nips.cc/paper/9451-shallow-rnn-accurate-time-series-classification-on-resource-constrained-devices.pdf)

Предложили RNN ячейку, которая может почти параллельно обрабатывать временной ряд, не теряя большую часть информации. Зафиксировали прирост производительности, почти не уронили рекуррентность. Метод показывает, что авторы - любители деревьев, потому что они построили дерево даже из RNN ячеек, разбивая исходный временной ряд на $\sqrt(T)$ частей и пропуская каждую часть через копию ShaRNN ячейки. Полученные $\sqrt(T)$ hidden state'ов подаются в следующую ShaRNN, и тд, пока длина ряда не будет достаточно малой для "эффективного вычисления". Дополнительно предлагают архитектуру ячейки MishaRNN

Авторы верят в (почти что) мультипликативность hidden state'ов RNN ячеек, их эксперименты показывают повышение точности на стандартных датасетх. В целом статья похожа на изобретение residual connection'ов для RNN

# Выбранная статья

Я выбрал статью Gradien Isolated Learning Representations (GIL), потому потому что:

1. Она мне понравилась больше остальных
2. Её реализация не была полностью понятна на момент второго прочтения, хотелось взять статью посложнее, чтобы показать, что могу разобраться в сложной теме
3. Её реализация не была простой (статьи ShaRNN и GAN SE показались мне менее техническими сложными для реализации, они умещались в шаблон class Model, for e in range(epochs), loss = criterion())
4. Она казалась мне релевантной для указанной темы стажировки (speaker diarization)


### Подробнее почему релевантна для стажировки

Во-первых, unsupervised - это всегда круто, потому что обычно задачи приходят в виде "ну у нас нет данных, но мы хотим чтобы работало вот так", а в случае соцсети полезных данных много, но они не размечены. Репрезентации данных значительно упрощают построение ML-моделей поверх них, это упрощает решение практических задач

Во-вторых, этот подход реализует не конкретную задачу (как удаление шумов на аудио), а даёт репрезентацию голоса, что можно применить в разных проектах. Поэтому однажды обученная модель может пригодиться в разных задачах, это даёт простор для выбора интересного исследования. 

В-третьих, в статье сделан упор на экономию вычислительных ресурсов, что всегда важно при экспериментах на полных данных (их может быть очень много, а вот видеокарт много не бывает), а также для меня, потому что мои 2ГБ видеопамяти в ноуте давно заставляют меня урезать модельки, стараясь не уронить их качество. Увидев слова "have reduced memory consumtion by factor 2.8", я подумал, что обойдусь без CUDA out of memory. Итеративное обучение модели тоже показалось мне отличной экономией ресурсов, ведь если вышестоящие модули не учатся в данный момент, то зачем прогонять на них данные? Вычисления на полное обучение сокращаются в 2 раза при таком подходе

 ---

# О методе

### Суть и особенности

Сначала о методе Contrastive Prediction Coding ([CPC](https://arxiv.org/pdf/1807.03748.pdf)). Предлагается end-to-end создание эмбеддинга для фрагмента аудио. У хорошо обученной модели эмбеддинг должен нести информацию о звуках аудио и о характере голоса спикера. Исходная аудиодорожка сворачивается до меньшей размерности с помощью $g_{enc}$, получается представление небольшой части аудио, назовём его $z_t$. Мы хотим, чтобы $z$ хранила общую для всего аудиоотрывка информацию, а не только локальную. Мы должны увеличить взаимную информацию $z_t$ со всеми t этой аудиодорожки.

Для этого используем RNN ячейку - в своём hidden state она протащит контекстную информацию, совместив её с текущим патчем данных. Подогнав размерности RNN, мы получим аналогичную $z_t$ репрезентацию, но уже совмещённую с прошлой информацией.

Чтобы выделять отличительные особенности аудио, будем пытаться предугадать следующие $z_{t+k}$, исходя из $z_t$, как показано на рисунке. С помощью некоторого преобразования мы попытаемся оценить $z_{t+k}$. Во время обучения модель будет стараться уменьшить расхождение между своей оценкой и действительными будущими z. 

Предугадывание помогает научиться лучше улавливать паттерны звука, это повышает точность на задаче классификации звука (в статье CPC показано падение точности при уменьшении дальности предугадывания). Чтобы решить вторую задачу - определение спикера - добавляется Noise Contrastive Estimation (NCE). Это выкрученный на максимум негативный сэплинг аудиофрагментов для логистической регрессии, встроенный прямо в модель. Учась отличать текущее аудио от негативных примеров, модель учится улавливать особенности самого голоса на аудио.

![CPC architecture](report_pictures/cpc_arch.png)

Теперь о Grad-Isolated. Авторы распилили монолитную архитектуру CPC на 5 модулей conv-relu-bn, добавив в каждый NCE. Таким образом, каждый модуль может сам себя обучать засчёт NCE. По мере приближения модуля к сходимости, он научится строить репрезентации, исходя из данных предыдущего модуля. Такой подход обеспечивает жадность при построении репрезентаций, делая репрезентации последнего модуля высокоуровневыми и позволяя им хранить максимум информации в минимуме места.

Независимость модулей наводит на мысль о том. чтобы полностью их изолировать. Для этого создан gradient-block, который обрезает градиенты между модулями. Теперь модули точно-точно учатся только сами в себе. Исходя из независимости модулей, мы можем тренировать их поэтапно, обучая только один из них, не используя градиенты в нижележащих модулях и не запуская вышестоящие. Это позволяет оптимизировать потребление ресурсов (и cpu, и ram)

### Полезность

Модули можно обучать итеративно, отсутсвтие градиентов в нижележащих модулях освободит ресурсы для больших batchsize'ов и negative sample'ов, что повысит качество модели и освободит место для обучения других моделей на видеокартах. Задача хорошо параллелится на несколько gpu.

Модульность архитектуры позволяет накинуть модулей сверху, чтобы получить ещё более сжатые репрезентации или охватить более длинную часть аудиофрагмента

 ---
 

# Что я сделал

1. Реализовал модель с Gradient Isolated Learning (GIL)
2. Реализовал модель с Contrastive Prediction Coding
3. Оптимизировал Negative Sampling для обеих моделей
4. Отделил конфигурации для экспериментов от моделей
5. Протестировал подход на задачах классификации
6. Не заимствовал какой-либо код, кроме торча и sklearn'а

Линия раздела времени 

6. Построил T-SNE диаграмму эмбеддингов спикеров
7. Показал результаты моделей
8. Показал графики обучения
9. Предложил использование модели
10. Предложил улучшение модели
11. Расшарил чекпоинты моделей на Google Drive

### Модели

В соответствующих папках лежат исходники моделей и стандартные конфиги, код для тренировки лежит в корне

- GIL - оптимизированная модель, которую предлагалась реализовать
- CPC_true_NCE - модель CPC с Negative Constrastive Estimation
- CPC - мой ablation study по CPC, модель не имеет негативного сэмплинга, её лосс изменён. Её результаты хуже оригинальной CPC

### Результаты GIL

Следуя настройкам из статьи, были использованы: итеративная тренировка, $k=12$ шагов предсказания RNN, Adam с lr $2e-4$, архитектура из 5 модулей, длина аудиофрагмента 20480 (что соответствует 1.28сек), но было взято 5 негативных сэмплов вместо 10, так как вычисления на видеокартах проводил не только я. По той же причине не получилось провести тренировку на 300 эпох - мой процесс убили на 38ой, а заметил я это довольно поздно :с

На иллюстрации показаны лоссы на train (сверху) и test (снизу) выборке (разбиение 0.2). На первый взгляд кажется, что модель оверфитнулась, но это не так. Если посмотреть на ось Y, то видно, что на train модель быстро сошлась к уровню 5.56, а на тесте, несмотря на видимый разброс, лосс колеблется на тысячных от значения 5.56. Интересно, что с каждым модулем растёт уровень лосса, к которому сходится модель. Для первого модуля это 5.56, для второго 5.7 и 5.61, 5.88 соответственно. Я думаю, что это связано с ошибками нижележащих слоёв. Так как последующие слои используют информацию преыдудщих как данные без возможности их оптимизации, то будут ошибаться на выбросах плохих репрезентаций.

![Лоссы модулей при итеративной тренировке GIL](report_pictures/gil_train_iteratively_losses.png)

Авторы статьи приводили графики лосса первых 3 модулей (красный - итеративная тренировка), они имеют сходну гиперболическую форму, как и в моих результатах (только у них 1000 эпох, а также другое количество neg sample'ов, поэтому графики ниже по Y), однако у авторов наблюдалось сильное расхождение между train и validation loss'ом на первых же модулях, в то время как в моём эксперименте расхождение было в последних двух (может быть вызваны недостаточным временем тренировки последних модулей. Я тренировал каждый модуль ~ 2 эпохи, это оказалось лучшим сеттингом)

![GIL loss авторов](report_pictures/gil_train_paper_losses.png)

что-то про спикеров

### Результаты CPC

Как и для GIL модели, сдледовал настройкам авторов статьи, лосс быстро сошёлся к одному уровню и незначительно падал в ходе следующих эпох. дополнительно логал точность каждого из $k=12$ предсказательных модулей. Авторы в статье говорили, что предсказательная способность модулей падает по гиперболе, точность после 5ого модуля уже ниже 0.3, но мои 12 модулей имели примерно одинаковую точность нахождения positive sample'а среди негативных. Это может быть связано с разным количеством негативных примеров в эксперименте авторов по сравнению со стандартной конфигурацией обучения, которую они предложили в статье

![train loss CPC](report_pictures/cpc_train_loss.png) | ![test loss CPC](report_pictures/cpc_test_loss.png)

Точность определения позитивного примера у $k$ модулей CPC модели держится на уровне 90% на тестовой выборке

![CPC test accuracy](report_pictures/cpc_test_accuracy.png)

что-то проспикеров

### Использование модели в продакшене

Модель может быть использована для выделения спикеров на аудио и (аудио взятом из) видео, что может перенести фичу упоминания людей на фотках на видео, голосовые сообщения и подкасты. Более того, можно не только указать, что человек есть на записи, но и указать интервал, на котором он говорит. Это может дать возможность поиска аудиоконтента по автору

Может быть использована для распознавания голоса владельца внутри приложения (какие-либо проекты с подтверждением транзакций биометрией посредством голоса / голосовым ассистентом / управлением без рук)

Модель не является самостоятельной - поверх неё нужно обучать классификаторы, что делает её полезной для разных задач

### Возможные улучшения и исследования

Теоретические
- Попробовать использовать репрезентации модели в распознавании речи (на звуках справлялась неплохо)
- Пробовать multitask learning на phone- и speaker-classification, мб ускорит/улучшит сходимость
- Попробовать attention в RNN (правда авторы сами указали такую возможность улучшения, не очень интересно)
- Попробовать перевернуть модель и засунуть в GAN, ведь все так делают, чтобы выпускать новые статьи, правда??


Технические
- Оптимизация негативного сэмплинга (оказалось, в реализации автора статьи для аккуратного сэмплинга используется медленный выбор noise примеров, реализовал)
- Изменение функции оценки $f_k()$ предсказываемых $z_{t+k}$ (сейчас это линейные слои, проовал 1d свёртки, результаты чуть хуже, можно попробовать навесить активаций или добавить слоёв с bottleneck'ом - должно улучшить выделение значимых признаков)
- Сделать повторные тренировки модулей (применить Scheduler'ы к разморозке/заморозке модулей, это может ускорить сходимость при итеративной и одновременной тренировке)